# Ansatzes

In [ ]:
import logging
logging.basicConfig(
    format='%(asctime)s-%(levelname)s: %(message)s',
    datefmt='%m/%d/%Y %I:%M:%S %p',
    level=logging.INFO
    #level=logging.DEBUG
)
logger = logging.getLogger('__name__')

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import sys
sys.path.append("../")

In [ ]:
# myQLM qpus
from qat.qpus import PyLinalg, CLinalg
qpu_c = CLinalg()
qpu_p = PyLinalg()

# QLM qpus
from qlmaas.qpus import LinAlg
qpu_qaass = LinAlg()
from qlmaas.qpus import MPS
qpu_mps = MPS()

## 1. ansatzes module

One mandatory step for using the Parent Hamiltonian, **PH**, library (see nootebook **02_Using_PH_Class.ipynb**) is computing for a given ansatz its complete state. This is the amplitudes of the state in the computational *n* qubit basis. 

In the *ansatzes* module of the **PH** library several functions and classes for deling with this part of the computation was done.


### 1.1 SolveCircuit class

The **SolveCircuit** takes a *Atos myqlm circuit* with an ansatz, fix their parameter and simulates using *Atos qpu* and return the state of the ansatz. This class needs for initialization following arguments:

* qlm_circuit: *Atos myqlm circuit* with an ansatz and wit the parameters fixed
* kwargs: keyword arguments where following keys are proccesed:
    * parameters: pandas DataFrame with the parameters of the circuit
    * qpu: Atos qpu for simulate the ansatz
    
For showing how this class works some example circuits will be provided

In [ ]:
from ansatzes import SolveCircuit

### 1.2 Parent Hamiltonian Github ansatz

The *ansatz_qlm_01* functions implements a **Atos myqlm** version of the ansatz in the github:

https://github.com/FumiKobayashi/Parent_Hamiltonian_as_a_benchmark_problem_for_variational_quantum_eigensolvers

from the original Parent Hamiltonian Papper:

* Kobayashi, F., Mitarai, K., & Fujii, K. (2022). Parent hamiltonian as a benchmark problem for variational quantum eigensolvers. Phys. Rev. A, 105, 052415 (https://doi.org/10.1103%2Fphysreva.105.052415)

We need functions *ansatz_qlm_01* and the function *solve_ansatz* that allows the simulation of a **Atos myqlm** program

In [ ]:
from ansatzes import ansatz_qlm_01

We need to provided to the *ansatz_qlm_01* the number of desired qubits and the depth of the circuit. The function returns an *Atos myqlm circuit* of the ansatz.

In [ ]:
n_qubits = 12
depth = 2
circ_ansatz01 = ansatz_qlm_01(nqubits=n_qubits, depth=depth)

In [ ]:
%qatdisplay circ_ansatz01 --svg

### Setting the parameters

As can be seen the circuit has the parameters as variables. We can set the parameters calling the **angles_ansatz01**function. If $n_l$ is the number of layers of the circuit the formula for setting the parameters is:


$$\delta \theta = \frac{\pi}{4*(n_l+1)}$$

$$\theta_i = (i+1) \delta \theta \; i=0, 1, \cdots 2n_l-1$$

The output of the function are:
* circuit with the parameters fixed
* pandas DataFrame with the value of the parameters used

In [ ]:
from ansatzes import angles_ansatz01

In [ ]:
circ_ansatz01_, pdf_par = angles_ansatz01(circ_ansatz01)

In [ ]:
%qatdisplay circ_ansatz01_ --svg

In addition we can use the parameters of the circuit providing a pandas DataFrame to the function. In this case the function can be used for setting the parameters for any circuit! The parameters should be passes as a pandas DataFrame with 2 columns:
* key: string with the name of the parameter (the same name that has in the circuit)
* value: float value of the correspondent parameter

In [ ]:
parameters = {v_ : 2 * np.pi * np.random.rand() for i_, v_ in enumerate(circ_ansatz01.get_variables())}
angles = [k for k, v in parameters.items()]
values = [v for k, v in parameters.items()]
pdf_parameters = pd.DataFrame([angles, values], index=['key', 'value']).T

In [ ]:
pdf_parameters

In [ ]:
circ_ansatz01_2, _ = angles_ansatz01(circ_ansatz01, pdf_parameters)

In [ ]:
# The output is the circuit with the parameters fixed
%qatdisplay circ_ansatz01_2 --svg

In [ ]:
from utils import create_folder

filename = "ansatz_{}_dept_{}_nqubits_{}".format("simple01", depth, n_qubits)
folder = create_folder("kk")

class_dict = {
    "qpu" : qpu_c,
    "parameters" : pdf_parameters,
    "filename": folder + filename,
    "save": True
}

In [ ]:
solv_ansatz01 = SolveCircuit(circ_ansatz01_, **class_dict)

In [ ]:
solv_ansatz01.run()

For simulating the ansatz the *run* method of the class should be executed when properly configuration is done. The *state* attribute will contain a pandas DataFrame with complet state information

In [ ]:
solv_ansatz01.state

### 1.2 Parent Hamiltonian General ansatz

The function *ansatz_qlm_02* implements a generalization of the *ansatz_qlm_01* one. In the *ansatz_qlm_01* all the qubits has the same operations with the same parameters. In the *ansatz_qlm_02* each qubit has the same operations but each operation will have a different  parameter.

The *SolveCircuit* class can be used for solving the ansatz

In [ ]:
from ansatzes import ansatz_qlm_02

In [ ]:
n_qubits = 8
depth = 3
circ_ansatz02 = ansatz_qlm_02(nqubits=n_qubits, depth=depth)

In [ ]:
%qatdisplay circ_ansatz02 --svg

In [ ]:
filename = "ansatz_{}_dept_{}_nqubits_{}".format("simple02", depth, n_qubits)
folder = create_folder("kk")

# Create the parameter values
parameters = {v_ : 2 * np.pi * np.random.rand() for i_, v_ in enumerate(circ_ansatz02.get_variables())}
angles = [k for k, v in parameters.items()]
values = [v for k, v in parameters.items()]
pdf_parameters = pd.DataFrame([angles, values], index=['key', 'value']).T


In [ ]:
circ_ansatz02_, _ = angles_ansatz01(circ_ansatz02, pdf_parameters)

In [ ]:
class_dict = {
    "qpu" : qpu_c,
    "parameters" : pdf_parameters,
    "filename": folder + filename,
    "save": True    
}
solv_ansatz02 = SolveCircuit(circ_ansatz02_, **class_dict)
solv_ansatz02.run()

In [ ]:
solv_ansatz02.state

### 1.3 Other ansatzes

For using the **PH** library for computing Parent Hamiltonians the mandatory input will be the list of amplitudes. We can use different ansatzes that are avialable from *Atos myqlm* library.

In [ ]:
from qat.fermion.circuits import make_ldca_circ, make_general_hwe_circ

In [ ]:
nqubit = 8
depth = 3
lda_circ = make_ldca_circ(nqubit, depth)

In [ ]:
%qatdisplay lda_circ --svg

In [ ]:
filename = "ansatz_{}_dept_{}_nqubits_{}".format("ldca", depth, nqubit)
folder = create_folder("kk")

# Create the parameter values
parameters = {v_ : 2 * np.pi * np.random.rand() for i_, v_ in enumerate(lda_circ.get_variables())}
angles = [k for k, v in parameters.items()]
values = [v for k, v in parameters.items()]
pdf_parameters = pd.DataFrame([angles, values], index=['key', 'value']).T

In [ ]:
lda_circ_, _ = angles_ansatz01(lda_circ, pdf_parameters)

In [ ]:
%qatdisplay lda_circ_ --svg

In [ ]:
class_dict = {
    "qpu" : qpu_c,
    "parameters" : pdf_parameters,
    "filename": folder + filename,
    "save": True    
}
solv_ldca = SolveCircuit(lda_circ_, **class_dict)
solv_ldca.run()

In [ ]:
solv_ldca.state

In [ ]:
hwe_circ = make_general_hwe_circ(nqubit, n_cycles=1)

In [ ]:
%qatdisplay hwe_circ --svg

In [ ]:
filename = "ansatz_{}_dept_{}_nqubits_{}".format("hwe", 1, nqubit)
folder = create_folder("kk")
# Create the parameter values
parameters = {v_ : 2 * np.pi * np.random.rand() for i_, v_ in enumerate(hwe_circ.get_variables())}
# Create the parameter values
parameters = {v_ : 2 * np.pi * np.random.rand() for i_, v_ in enumerate(lda_circ.get_variables())}
angles = [k for k, v in parameters.items()]
values = [v for k, v in parameters.items()]
pdf_parameters = pd.DataFrame([angles, values], index=['key', 'value']).T
hwe_circ_, _ = angles_ansatz01(hwe_circ, pdf_parameters)

In [ ]:

class_dict = {
    "qpu" : qpu_c,
    "parameters" : pdf_parameters,
    "filename": folder + filename,
    "save": True    
}
solv_hwe = SolveCircuit(hwe_circ_, **class_dict)
solv_hwe.run()

In [ ]:
%qatdisplay hwe_circ_ --svg

In [ ]:
solv_hwe.state

### 1.4 Ansatz selector

In order to simplify the ansatz selection a function called **ansatz_selector** was built. 

In [ ]:
from ansatzes import ansatz_selector

In [ ]:
conf_dict = {
    'nqubits' : 4,
    'depth' : 20
}

In [ ]:
circuit = ansatz_selector('simple01', **conf_dict)
%qatdisplay circuit --svg

In [ ]:
parameters = {v_ : 2 * np.pi * np.random.rand() for i_, v_ in enumerate(circuit.get_variables())}

In [ ]:
circuit = circuit(**parameters)

In [ ]:
%qatdisplay circuit --svg

In [ ]:
circuit = ansatz_selector('simple02', **conf_dict)
%qatdisplay circuit --svg

In [ ]:
circuit = ansatz_selector('lda', **conf_dict)
%qatdisplay circuit --svg

In [ ]:
circuit = ansatz_selector('hwe', **conf_dict)
%qatdisplay circuit --svg